In [19]:
from elasticsearch import Elasticsearch
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.axes_style("darkgrid")
sns.color_palette("colorblind", 10)
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

In [20]:
es = Elasticsearch([{'host': 'lame14.enst.fr', 'port': 50014}])
#index_name = 'tweets-2016.02.16'

In [21]:
#
# Retrouver la date des info trafic les plus récentes
#
query_max_date = {
 "aggs" : {
      "max_id" : {
        "max" : { 
          "field" : "pub_date"
        }
      }
    },
    "size":0
}
res = es.search(index="stif_info_trafic", body=query_max_date)
last_refresh = res[u'aggregations'][u'max_id'][u'value']
last_refresh

1455912786000.0

In [22]:
import re
from datetime import datetime, timedelta

#
# Afficher le contenu du dernier bulletin
#
query_info_trafic = {
    "fields" : ["_source"],
    "query" : {
        "term" : { "pub_date" : last_refresh }
    }
}

query_select_infos = {
    "fields" : ["_source"],
    "query" : {
            "range" : {
                "pub_date" : {
            "gte": "17/02/2016",
            "lte": "19/02/2016",
            "format": "dd/MM/yyyy"
                }
            }
        }
}

def clean_data(data):
    TAG_RE = re.compile(r'<[^>]+>')
    data['texte'] = TAG_RE.sub('', data['texte'])
    data['texte'] = data['texte'].replace('\n', ' ')
    data['description'] = data['description'].replace('\n', ' ')

TODAY = datetime.now()
YESTERDAY = TODAY - timedelta(days=1)
TODAY = datetime.strftime(TODAY, '%d/%m/%Y')
YESTERDAY = datetime.strftime(YESTERDAY, '%d/%m/%Y')
query_select_infos['query']['range']['pub_date']['gte'] = YESTERDAY
query_select_infos['query']['range']['pub_date']['lte'] = TODAY
res = es.search(index="stif_info_trafic", body=query_select_infos, size=100, from_=0)
# print res
bulletin = res[u'hits'][u'hits']
bulletin = map(lambda x: x['_source'], bulletin)
map(clean_data, bulletin)
data = pd.DataFrame(bulletin)
data

description  \
0   Axe concerné : Creil/Pontoise Train supprimé d...   
1   Jeudi 18 février 2016 à 06h26, en raison d'un ...   
2   Jeudi 18 février 2016 à 06h14, en raison d'un ...   
3   Jeudi 18 février 2016 à 06h04, en raison d'un ...   
4   Axe(s) concerné(s) : Paris/Mitry Claye Paris/A...   
5   Axe(s) concerné(s) : Paris/Robinson Paris/Sain...   
6   Jeudi 18 février 2016 à 08h58, en raison de me...   
7   Axe(s) concerné(s) : Paris Haussmann Saint Laz...   
8   Axe concerné : Bondy - Aulnay sous Bois. Prévo...   
9   Axe(s) concerné(s) : Paris/Pontoise/Gisors Pré...   
10  Axe concerné : Bondy - Aulnay sous Bois. Prévo...   
11  Jeudi 18 février 2016 à 07h04, sur la ligne 2 ...   
12  Axe concerné : Creil/Pontoise Train supprimé d...   
13  Axe(s) concerné(s) : Paris/Mitry Claye Paris/A...   
14  Axe concerné : Creil/Pontoise Train supprimé :...   
15  Axe concerné : Paris/Provins. Prévoir jusqu'à ...   
16  Axe concerné : Creil/Pontoise Train supprimé e...   
17  Jeudi 18 février 2016 à 07h09, en raison d'un ...   
18  Axe concerné : Bondy - Aulnay sous Bois. Prévo...   
19  Jeudi 18 février 2016 à 06h46, en raison d'un ...   
20  Axe(s) concerné(s) : Paris/Pontoise/Gisors Pré...   
21  Axe concerné : Paris / Marne la Vallée Chessy ...   
22  Axe(s) concerné(s) : Paris Haussmann Saint Laz...   
23  Axe(s) concerné(s) : Paris/Robinson Paris/Sain...   
24  Axe(s) concerné(s) : Paris Haussmann Saint Laz...   
25  Jeudi 18 février 2016 à 12h35, en raison de me...   
26  Jeudi 18 février 2016 à 11h42, en raison de me...   
27  Jeudi 18 février 2016 à 12h17, en raison de me...   
28  Jeudi 18 février 2016 à 12h37, en raison d'un ...   
29  Jeudi 18 février 2016 à 10h38, en raison d'un ...   
30  Axe(s) concerné(s) : Paris/Dourdan la Forêt Pa...   
31  Axe concerné : Paris/Persan Beaumont via Valmo...   
32  Axe concerné : Paris/Persan Beaumont via Valmo...   
33  Axe concerné : Paris/Persan Beaumont via Valmo...   
34  Axe(s) concerné(s) : Paris/Dourdan la Forêt Pa...   
35  Axe concerné : Paris/Persan Beaumont via Valmo...   
36  Jeudi 18 février 2016 à 14h37, en raison d'un ...   
37  Jeudi 18 février 2016 à 14h10, en raison d'une...   
38  Axe(s) concerné(s) : Paris/Dourdan la Forêt Pa...   
39  Axe concerné : Paris/Gisors. Train supprimé : ...   
40  Jeudi 18 février 2016 à 17h23, en raison d'une...   
41  Jeudi 18 février 2016 à 17h27, en raison d'une...   
42  Axe(s) concerné(s) : Melun / Montereau via Hér...   
43  Axe(s) concerné(s) : Melun / Montereau via Hér...   
44  Axe concerné : - Paris / Cergy le Haut Prévoir...   
45  Axes concernés : Paris/Cergy le Haut Paris/Poi...   
46  Axe(s) concerné(s) : Paris/Dourdan la Forêt Pa...   
47  Axes concernés : Ensemble de la ligne. Le traf...   
48  Axe(s) concerné(s) : Melun / Montereau via Hér...   
49  Axe concerné : Paris/Gisors. Train supprimé : ...   
50  Axe(s) concerné(s) : Melun / Montereau via Hér...   
51  Axes concernés : Ensemble de la ligne. Le traf...   

                     pub_date  \
0   Thu, 18 Feb 2016 06:55:04   
1   Thu, 18 Feb 2016 06:30:05   
2   Thu, 18 Feb 2016 06:20:10   
3   Thu, 18 Feb 2016 06:10:06   
4   Thu, 18 Feb 2016 09:05:16   
5   Thu, 18 Feb 2016 09:35:13   
6   Thu, 18 Feb 2016 09:05:25   
7   Thu, 18 Feb 2016 09:05:23   
8   Thu, 18 Feb 2016 08:15:03   
9   Thu, 18 Feb 2016 08:10:07   
10  Thu, 18 Feb 2016 08:00:05   
11  Thu, 18 Feb 2016 07:10:05   
12  Thu, 18 Feb 2016 07:10:04   
13  Thu, 18 Feb 2016 08:40:08   
14  Thu, 18 Feb 2016 07:50:04   
15  Thu, 18 Feb 2016 07:40:04   
16  Thu, 18 Feb 2016 07:20:05   
17  Thu, 18 Feb 2016 07:15:04   
18  Thu, 18 Feb 2016 07:00:04   
19  Thu, 18 Feb 2016 06:50:04   
20  Thu, 18 Feb 2016 06:40:05   
21  Thu, 18 Feb 2016 06:20:07   
22  Thu, 18 Feb 2016 09:55:09   
23  Thu, 18 Feb 2016 09:45:12   
24  Thu, 18 Feb 2016 10:05:08   
25  Thu, 18 Feb 2016 12:40:06   
26  Thu, 18 Feb 2016 11:50:04   
27  Thu, 18 Feb 2016 12:25:04   
28  Thu, 18 Feb 2016 12:40:05   
29  Thu, 18 Feb 2016 10:45:04 

In [23]:
#
# On garde la date de publication et un texte composé du titre et du champ texte
#
final_data = data[['pub_date', 'texte']]
final_data['texte'] = final_data['texte'].str.replace('\n',' ')
final_data['texte'] = data['titre'] + '. ' + final_data['texte']
final_data['pub_date'] = pd.to_datetime(final_data['pub_date'], format="%a, %d %b %Y %H:%M:%S")
final_data

pub_date                                              texte
0  2016-02-18 06:55:04  Ligne H : suppressions Creil/Pontoise. Axe con...
1  2016-02-18 06:30:05  Métro 14 : Incident technique > Trafic interro...
2  2016-02-18 06:20:10  Métro 14 : Incident technique > Trafic interro...
3  2016-02-18 06:10:06  Métro 14 : Incident technique > Trafic interro...
4  2016-02-18 09:05:16  RER B : Paris/Mitry/Aéroport CDG 2 ralenti.. A...
5  2016-02-18 09:35:13  RER B : Paris/Robinson Saint-Rémy ralenti.. Ax...
6  2016-02-18 09:05:25  Métro 4 : Mesure de sécurité > Trafic retardé....
7  2016-02-18 09:05:23  RER E : Haussmann St Lazare non desservi. Axe(...
8  2016-02-18 08:15:03  Ligne T4 : trafic ralenti. Axe concerné : Bond...
9  2016-02-18 08:10:07  Ligne J : Paris/Pontoise/Gisors ralenti. Axe(s...
10 2016-02-18 08:00:05  Ligne T4 : trafic ralenti. Axe concerné : Bond...
11 2016-02-18 07:10:05  Métro 2 : Incident voyageur > Rame stationnée....
12 2016-02-18 07:10:04  Ligne H : suppressions Creil/Pontoise. Axe con...
13 2016-02-18 08:40:08  RER B : Paris/Mitry/Aéroport CDG 2 ralenti.. A...
14 2016-02-18 07:50:04  Ligne H : suppression Persan/Pontoise. Axe con...
15 2016-02-18 07:40:04  Ligne P : Paris/Provins ralenti. Axe concerné ...
16 2016-02-18 07:20:05  Ligne H : rectificatif suppressions Creil/Pont...
17 2016-02-18 07:15:04  Métro 2 : Incident voyageur > Trafic retardé. ...
18 2016-02-18 07:00:04  Ligne T4 : trafic ralenti. Axe concerné : Bond...
19 2016-02-18 06:50:04  Métro 14 : Incident technique > Trafic retardé...
20 2016-02-18 06:40:05  Ligne J : Paris/Pontoise/Gisors ralenti. Axe(s...
21 2016-02-18 06:20:07  RER A : Paris / Boissy Marne la Vallée ralenti...
22 2016-02-18 09:55:09  RER E : Haussmann St Lazare non desservi. Axe(...
23 2016-02-18 09:45:12  RER B : Paris/Robinson Saint-Rémy ralenti.. Ax...
24 2016-02-18 10:05:08  RER E : Haussmann St Lazare non desservi. Axe(...
25 2016-02-18 12:40:06  Métro 9 : Mesure de sécurité > Trafic retardé....
26 2016-02-18 11:50:04  Métro 9 : Mesure de sécurité > Trafic interrom...
27 2016-02-18 12:25:04  Métro 9 : Mesure de sécurité > Arrêt non marqu...
28 2016-02-18 12:40:05  Tramway T3a : Malaise voyageur > Trafic interr...
29 2016-02-18 10:45:04  Métro 6 : Incident technique > Trafic retardé....
30 2016-02-18 16:00:23  RER C : Paris/Dourdan/Etampes/Massy ralenti. A...
31 2016-02-18 14:50:05  Ligne H : Valmondois / Ermont Eaubonne interro...
32 2016-02-18 15:50:09  Ligne H : Valmondois / Saint-Leu interrompu. A...
33 2016-02-18 15:40:06  Ligne H : Valmondois / Saint-Leu interrompu. A...
34 2016-02-18 15:20:15  RER C : Paris/Dourdan/Etampes/Massy ralenti. A...
35 2016-02-18 15:10:07  Ligne H : Valmondois / Saint-Leu interrompu. A...
36 2016-02-18 14:40:04  Métro 6 : Malaise voyageur > Trafic retardé. J...
37 2016-02-18 14:15:05  Métro 9 : Manifestation > Station fermée compl...
38 2016-02-18 16:20:19  RER C : Paris/Dourdan/Etampes/Massy ralenti. A...
39 2016-02-18 17:55:11  Ligne J : suppression Paris/Gisors. Axe concer...
40 2016-02-18 17:30:05  Métro 7 : Panne de matériel > Trafic retardé. ...
41 2016-02-18 17:30:04  Métro 6 : Panne de matériel > Trafic retardé. ...
42 2016-02-18 19:35:05  Ligne R : Montereau/Melun via Héricy interromp...
43 2016-02-18 19:25:04  Ligne R : Montereau/Melun via Héricy interromp...
44 2016-02-18 18:35:11  Ligne L : Paris/Cergy le Haut ralenti. Axe con...
45 2016-02-18 18:35:07  RER A : Paris/Cergy le Haut/Poissy ralenti. Ax...
46 2016-02-18 17:10:23  RER C : Paris/Dourdan/Etampes/Massy ralenti. A...
47 2016-02-18 11:15:19  RER B : trafic ralenti.. Axes concernés : Ense...
48 2016-02-18 20:10:04  Ligne R : Montereau/Melun via Héricy interromp...
49 2016-02-18 18:05:24  Ligne J : suppression Paris/Gisors. Axe concer...
50 2016-02-18 20:50:04  Ligne R : Montereau/Melun via Héricy interromp...
51 2016-02-18 17:35:40  RER B : trafic ralenti.. Axes concernés : Ense...

In [24]:
#
# Définition de la marge de recherche qu'on va utiliser autour des dates de publication
#
from datetime import datetime
from datetime import timedelta

time_delta_minus =  timedelta(hours=-1)
time_delta_plus =  timedelta(hours=1)

#
# On recherche dans les index de la veille, du jour et du lendemain
#
def get_tweet_index_name(date_time, day_delta):
    index_date = date_time
    if day_delta > 0:
        index_date += timedelta(days=1)
    elif day_delta < 0:
        index_date += timedelta(days=-1)
    # retrieve index suffix
    index_suffix = index_date.strftime("%Y.%m.%d")
    return "tweets-" + index_suffix

#
# Définir un intervalle autour d'une date
#
def get_bounds_dates(date_time, margin):
    index_date1 = date_time
    index_date2 = date_time
    index_date1 += timedelta(hours=-margin)
    index_date2 += timedelta(hours=margin)
    return index_date1, index_date2

# TU
# for i in range(0, len(final_data)):
#     dt = final_data['pub_date'][i]
#     print dt, get_tweet_index_name(dt, -1), get_tweet_index_name(dt, 0), get_tweet_index_name(dt, 1)

In [25]:
#
# Retrieve date in milliseconds since EPOCH
#
def get_time(date_time):
    EPOCH = datetime.strptime('19700101', '%Y%m%d')
    diff = date_time - EPOCH
    return (diff.days*24*3600 + diff.seconds)*1000 + diff.microseconds/1000

print get_time(datetime.now())

1455923901385


In [26]:
import pystache
import json

#
# Afficher le contenu du dernier bulletin
#
query_sel_tweets = '{ \
"fields": ["_source"], \
 "query": { \
    "bool": { \
      "must": [ \
        { \
          "match": { \
            "user.time_zone": "Paris" \
          } \
        }, \
       { \
       "range": { \
             "created_at": { \
               "gte": "{{dt_min}}", \
               "lte": "{{dt_max}}", \
               "format": "yyyyMMddHHmmss" \
             } \
           } \
       } \
       ] \
    } \
  } \
}'

# query_select_tweets = {
#     "fields" : ["_source"],
#     "query" : {
#             "range" : {
#                 "created_at" : {
#                     "gte" : "",
#                     "lte" : "",
#                     "format": "yyyyMMddHHmmss"
#                 }
#             }
#         }
# }

def read_tweet_page(index_name, size, from_, query):
    res = es.search(index=index_name, body=query, size=size, from_=from_)
    return res

# import traceback

#
# Recherche de tweets autour d'une date donnée
#
def lookup_tweets(date_time, margin):
    data = None
    pg_size = 1000
    try:
        dt_min, dt_max = get_bounds_dates(date_time, 1)
        index_name = get_tweet_index_name(date_time, margin)
        dt_min = datetime.strftime(dt_min, '%Y%m%d%H%M%S')
        dt_max = datetime.strftime(dt_max, '%Y%m%d%H%M%S')
        # print dt_min, dt_max
        query_select_tweets['query']['range']['created_at']['gte'] = dt_min
        query_select_tweets['query']['range']['created_at']['lte'] = dt_max
        q = json.loads(pystache.render(query_sel_tweets, {'dt_min': dt_min, 'dt_max': dt_max}))
        res = read_tweet_page(index_name, pg_size, 0, q)
        i = 1
        data = []
        while len(res[u'hits'][u'hits']) == pg_size:
            res = map(lambda x: [x['_source']['created_at'], x['_source']['text']], res[u'hits'][u'hits'])
            data.extend(res)
            res = read_tweet_page(index_name, pg_size, i*pg_size, query_select_tweets)
#             print len(data)
            i += 1
        if len(res[u'hits'][u'hits']) > 0:
            res = map(lambda x: [x['_source']['created_at'], x['_source']['text']], res[u'hits'][u'hits'])
            data.extend(res)
#             print len(data)
    except:
        # traceback.print_exc()
        pass
    return data


# def view_bulletin(rank):
#     if rank < len(bulletin):
#         print "-----------------------------------------------------"
#         dt = datetime.strptime(bulletin[rank]['pub_date'], "%a, %d %b %Y %H:%M:%S")
#         print dt, bulletin[rank]['texte']
#         print
#         for margin in [-1, 0, 1]:
#             data = lookup_tweets(dt, margin)
#             if data is not None:
#                 for d in data:
#                     print d


In [27]:
rank = 0
print "-----------------------------------------------------"
dt = datetime.strptime(bulletin[rank]['pub_date'], "%a, %d %b %Y %H:%M:%S")
print dt, bulletin[rank]['texte']
print

data = lookup_tweets(dt, -1)
# print data
pd.DataFrame(data)

-----------------------------------------------------
2016-02-18 06:55:04 Axe concerné : Creil/Pontoise Train supprimé de Creil à Persan Beaumont, circulant au départ de Persan Beaumont : - (Creil 06h50) - Persan Beaumont 07h19 - Pontoise 07h49 - mission TOLI. Train supprimé sur la totalité de son parcours :  - Pontoise 07h05 - Persan Beaumont 07h43 - Creil 08h06 - mission TSOL. Motif : avarie de matériel. SNCF TransilienCette information concerne 			



0  \
0    Thu Feb 18 06:01:00 +0000 2016   
1    Thu Feb 18 06:08:28 +0000 2016   
2    Thu Feb 18 06:10:17 +0000 2016   
3    Thu Feb 18 06:07:08 +0000 2016   
4    Thu Feb 18 06:11:43 +0000 2016   
5    Thu Feb 18 06:18:38 +0000 2016   
6    Thu Feb 18 06:20:49 +0000 2016   
7    Thu Feb 18 06:22:40 +0000 2016   
8    Thu Feb 18 06:25:08 +0000 2016   
9    Thu Feb 18 06:31:16 +0000 2016   
10   Thu Feb 18 06:38:44 +0000 2016   
11   Thu Feb 18 06:36:42 +0000 2016   
12   Thu Feb 18 06:39:35 +0000 2016   
13   Thu Feb 18 06:43:04 +0000 2016   
14   Thu Feb 18 06:44:18 +0000 2016   
15   Thu Feb 18 06:49:58 +0000 2016   
16   Thu Feb 18 06:57:25 +0000 2016   
17   Thu Feb 18 06:51:09 +0000 2016   
18   Thu Feb 18 06:57:48 +0000 2016   
19   Thu Feb 18 07:00:56 +0000 2016   
20   Thu Feb 18 07:03:35 +0000 2016   
21   Thu Feb 18 07:03:54 +0000 2016   
22   Thu Feb 18 07:00:55 +0000 2016   
23   Thu Feb 18 07:01:46 +0000 2016   
24   Thu Feb 18 07:07:47 +0000 2016   
25   Thu Feb 18 07:05:23 +0000 2016   
26   Thu Feb 18 07:10:10 +0000 2016   
27   Thu Feb 18 07:13:19 +0000 2016   
28   Thu Feb 18 07:12:03 +0000 2016   
29   Thu Feb 18 07:12:57 +0000 2016   
..                              ...   
688  Thu Feb 18 07:21:06 +0000 2016   
689  Thu Feb 18 07:19:05 +0000 2016   
690  Thu Feb 18 07:20:23 +0000 2016   
691  Thu Feb 18 07:21:40 +0000 2016   
692  Thu Feb 18 07:24:08 +0000 2016   
693  Thu Feb 18 07:23:08 +0000 2016   
694  Thu Feb 18 07:27:05 +0000 2016   
695  Thu Feb 18 07:24:58 +0000 2016   
696  Thu Feb 18 07:32:05 +0000 2016   
697  Thu Feb 18 07:30:41 +0000 2016   
698  Thu Feb 18 07:30:39 +0000 2016   
699  Thu Feb 18 07:28:32 +0000 2016   
700  Thu Feb 18 07:30:14 +0000 2016   
701  Thu Feb 18 07:38:05 +0000 2016   
702  Thu Feb 18 07:36:46 +0000 2016   
703  Thu Feb 18 07:38:12 +0000 2016   
704  Thu Feb 18 07:38:32 +0000 2016   
705  Thu Feb 18 07:38:44 +0000 2016   
706  Thu Feb 18 07:39:17 +0000 2016   
707  Thu Feb 18 07:43:33 +0000 2016   
708  Thu Feb 18 07:42:05 +0000 2016   
709  Thu Feb 18 07:42:51 +0000 2016   
710  Thu Feb 18 07:41:21 +0000 2016   
711  Thu Feb 18 07:45:29 +0000 2016   
712  Thu Feb 18 07:43:38 +0000 2016   
713  Thu Feb 18 07:52:37 +0000 2016   
714  Thu Feb 18 07:53:12 +0000 2016   
715  Thu Feb 18 06:04:04 +0000 2016   
716  Thu Feb 18 06:30:28 +0000 2016   
717  Thu Feb 18 06:15:36 +0000 2016   

                                                     1  
0    ptn rien que j'vois des gens là ils sont dans ...  
1    RT @Alloprof: ESSAYEZ LE NOUVEL EXERCICE DE VO...  
2    La Hadopi ne peut plus repérer certains abonné...  
3    Kourtney Kardashian garde la ligne ! (Photos) ...  
4    En ce moment sur https://t.co/AKTSVd4FnW :\n A...  
5    RT @EvolvingSun: "@InSunWeTrust: 1ère station ...  
6    J'ai eu mon bus grave à l'arrache encore ptn j...  
7    dans 2 minutes je dois partir pour aller à l'a...  
8                 Les petits qui parlent dans le bus 😤  
9    Popcorn Time remis en ligne pour traquer ses v...  
10   RT @LaTribune: La banque en ligne se démocrati...  
11   #Montbéliard\nDéjà à l'origine d'esclandres à ...  
12   En ce moment sur Antilles Dom Station radio Ke...  
13   RT @MINES_ParisTech: .@InSunWeTrust, l’applica...  
14   En ce moment sur Antilles Dom Station radio So...  
15   Mais si les apparences sont trompeuses,St Thom...  
16   RT @Valmorel_alps: De la neige, du soleil et d...  
17   Et en #Rerb, 🚂 t'as carrément le temps de tout...  
18   Économie de proximité - Transdev veut accélére...  
19   RT @fcinq: Ce papier à la gare routière de Bey...  
20   @RERB @ratp gros problème ce matin sur le #rer...  
21   3 outils en ligne pour créer des vidéos intera...  
22   Retour à un trafic régulier sur l'ensemble de ...  
23   quand la sncf t'annonce que ce matin "tout va ...  
24   RT @Korben: Enlèvement colis UPS : notre chauf...  
25   Ça me dépassera toujours autant qu'un simple a...  
26   j'aime bien les chauffeurs de RER qui tape la ...  
27   C'est un peu c

In [47]:
#
# Construction d'un bag of words avec algorithme bm25
# on y place les alertes via navigo et les messages extraits de la base
#
from gensim.summarization.bm25 import BM25

# print bulletin
corpus_data = map(lambda x: x['titre']+' '+x['texte'], bulletin)
# print corpus_data
# construire le corpus ==>
corpus = " ".join([text for text in corpus_data
                            if 'http' not in text
#                                 and not word.startswith('@')
#                                 and text != 'RT' and text != '\n\r' and text != '\n'
                            ])
# print corpus

bm25 = BM25(corpus)

print data[0]
Query = data[0][1]
Query = Query.split()
scores = bm25.get_scores(Query, 0.9)
print scores
# tfidf = bm25.TFIDF()
# print bm25.Items()
# for i, tfidfscore in enumerate(tfidf):
#     print i, tfidfscore

[u'Thu Feb 18 06:01:00 +0000 2016', u"ptn rien que j'vois des gens l\xe0 ils sont dans leur bus et tout les pauvres, moi j'ai toujours pas commencer ma nuit"]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 